In [ ]:
import ee
import geemap
import math

# --- 1. Define Important Variables ---
print("--- Defining Key Variables ---")
gcp_project_id = 'mapbiomas-solos-workspace'
samples_asset_id = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/matriz-v010_0_10cm'
brazil_boundary_id = 'projects/mapbiomas-workspace/AUXILIAR/biomas_IBGE_250mil'
# Using SIRGAS 2000 / Brazil Albers (EPSG:5880) - More specific to Brazil [1, 2]
target_crs = 'EPSG:5880'
target_scale = 100  # Meters
# Max distance for Euclidean kernel (user specified 200km) [3, 4, 5]
max_search_distance_meters = 1000 # 1 km
# Bounding box size for Area of Interest (AOI)
aoi_bbox_radius_km = 100 # Results in a 100km box

print(f"Google Cloud Project: {gcp_project_id}")
print(f"Soil Samples Asset ID: {samples_asset_id}")
print(f"Brazil Boundary Asset ID: {brazil_boundary_id}")
print(f"Target CRS: {target_crs}")
print(f"Target Scale: {target_scale} meters")
print(f"Max Search Distance: {max_search_distance_meters} meters")
print(f"AOI Radius: {aoi_bbox_radius_km} km")

In [ ]:
# --- 2. Setup: Initialize Earth Engine ---
print("\n--- Initializing Earth Engine ---")
try:
    # Attempt initializing with the specified project
    ee.Initialize(project=gcp_project_id)
    print(f'Google Earth Engine Initialized Successfully for project: {gcp_project_id}')
except ee.EEException as e:
    print(f'Google Earth Engine initialization failed for project {gcp_project_id}: {e}')
    print('Attempting authentication and default initialization...')
    try:
        ee.Authenticate()
        ee.Initialize()
        print('Authenticated and initialized with default project.')
    except Exception as auth_e:
        print(f"Authentication/Initialization failed: {auth_e}")
        # Handle error appropriately, maybe exit
except Exception as e:
    print(f"An unexpected error occurred during initialization: {e}")
    # Handle other potential exceptions if necessary

In [ ]:
# --- 3. Load Data and Define Area of Interest (AOI) ---
print("\n--- Loading Data and Defining AOI ---")

# Load Soil Sample Points using the defined asset ID
try:
    samples_full = ee.FeatureCollection(samples_asset_id)
    print(f"Successfully loaded full soil samples dataset from: {samples_asset_id}")
    print(f"Total number of sample points: {samples_full.size().getInfo()}")
except Exception as e:
    print(f"Error loading soil samples from '{samples_asset_id}'. Error: {e}")
    # Handle error appropriately, maybe exit

# Load Brazil Boundary using the defined asset ID and dissolve it
try:
    brazil_fc = ee.FeatureCollection(brazil_boundary_id)
    # Dissolve the geometries if it's a multi-feature collection (e.g., biomes) [6]
    brazil_geometry = brazil_fc.geometry().dissolve(maxError=1)
    # Create a single feature for consistent use (optional but good practice)
    brazil = ee.FeatureCollection(ee.Feature(brazil_geometry, {}))
    print(f"Successfully loaded and dissolved Brazil boundary from: {brazil_boundary_id}")
except Exception as e:
    print(f"Error loading or dissolving Brazil boundary from '{brazil_boundary_id}': {e}")
    # Handle error appropriately, maybe exit

# --- Define AOI: 1000km Rectangle around Brazil's Center ---
# Get the centroid of the dissolved Brazil geometry
brazil_centroid = brazil_geometry.centroid(maxError=1)

# Buffer the centroid by the defined radius to get the desired diameter area [7, 8, 9, 10]
# Then get the bounding box of that circle, specifying the target CRS
buffer_radius_meters = aoi_bbox_radius_km * 1000
aoi_rectangle = brazil_centroid.buffer(buffer_radius_meters, maxError=1).bounds(proj=target_crs, maxError=1) # Using EPSG:5880
print(f"Defined {aoi_bbox_radius_km*2}km x {aoi_bbox_radius_km*2}km AOI rectangle around Brazil's centroid.")

# --- Filter Sample Points to AOI ---
# Filter points to only those within the AOI rectangle for efficiency [19]
samples_aoi = samples_full.filterBounds(aoi_rectangle)
print(f"Number of sample points within AOI: {samples_aoi.size().getInfo()}")

# --- Plot Initial Data within AOI ---
map1 = geemap.Map()
map1.centerObject(aoi_rectangle, 6) # Center map on the AOI
map1.addLayer(brazil, {'color': 'gray'}, 'Brazil Boundary (Dissolved)', False) # Show full boundary for context, initially off
map1.addLayer(aoi_rectangle, {'color': 'yellow', 'fillColor': '00000000'}, 'AOI Rectangle') # Show AOI boundary
map1.addLayer(samples_aoi, {'color': 'red', 'pointSize': 1}, 'Soil Sample Points (within AOI)')
print("Plotting initial data (AOI, filtered sample points)...")
display(map1)

In [ ]:
# --- 4. Rasterize Points within AOI ---
print("\n--- Rasterizing Points within AOI ---")
# Create an empty image, cast to byte
empty_image = ee.Image(0).byte()

# Paint the FILTERED sample points onto the image with value 1 [12, 13, 14]
# This image ('presence_image_painted') inherits the default projection (likely WGS84)
presence_image_painted = empty_image.paint(
    featureCollection=samples_aoi, # Use filtered points
    color=1 # Assigns the pixel value 1 where points exist
)

# Explicitly reproject the painted image to the target CRS and scale. [15, 16, 17, 6, 18, 19]
# THIS IS CRUCIAL for the accuracy of the distance calculation.
presence_image_proj = presence_image_painted.reproject(
    crs=target_crs, # Using EPSG:5880 now
    scale=target_scale
)

# Clip the projected presence image to the AOI rectangle [11, 20, 10]
# This ensures the distance calculation only considers this area.
# This 'presence_image_aoi' IS correctly projected and clipped.
presence_image_aoi = presence_image_proj.clip(aoi_rectangle)

# Optional: Unmask to ensure background is explicitly 0 (good practice) [21, 8]
presence_image_aoi = presence_image_aoi.unmask(0)

# --- Plot Rasterized Points (Visualization Only) ---
map2 = geemap.Map()
map2.centerObject(aoi_rectangle, 6)
map2.addLayer(aoi_rectangle, {'color': 'yellow', 'fillColor': '00000000'}, 'AOI Rectangle')
# --- VISUALIZATION WORKAROUND ---
# Display the UNPROJECTED painted image ('presence_image_painted'), clipped,
# for visualization to avoid the CRS parsing error in geemap display.
map2.addLayer(
    presence_image_painted.clip(aoi_rectangle), # Use UNPROJECTED image here
    {'palette': ['000000', 'FFFFFF'], 'min': 0, 'max': 1}, # Black background, White points
    'Rasterized Points (Painted, Visualization Only)'
)
print("Plotting rasterized points (painted image for visualization)...")
print("Note: The image used for the actual distance calculation IS reprojected and clipped.")
display(map2)

In [ ]:
# --- 5. Calculate Euclidean Distance, Convert to Integer, and Compute Statistics ---
print("\n--- Calculating Euclidean Distance within AOI ---")
# Define the Euclidean kernel using the predefined variable [1, 2, 3]
euclidean_kernel = ee.Kernel.euclidean(max_search_distance_meters, 'meters')

# Calculate Euclidean distance using the CORRECTLY PROJECTED and CLIPPED 'presence_image_aoi' [1, 4, 2, 3]
# skipMasked=False ensures distance is calculated for all pixels (0-value pixels) within the AOI [1, 4, 2, 3]
distance_image_aoi = presence_image_aoi.distance(
    kernel=euclidean_kernel,
    skipMasked=False
)

# Rename the output band for clarity
distance_image_aoi = distance_image_aoi.rename('distance_m')
print(f"Distance calculation initiated within AOI with max distance: {max_search_distance_meters} m")

# --- Convert Distance Image to Integer ---
# print("\n--- Converting Distance Image to Integer ---")
# Convert the floating-point distance image to integer [10]
# distance_image_aoi_int = distance_image_aoi_float.int()
# Rename the band to reflect integer type (optional but good practice)
# distance_image_aoi_int = distance_image_aoi_int.rename('distance_m_int')
# print("Distance image converted to integer type.")
# print image info
print(distance_image_aoi.getInfo())
# Get the maximum distance value for statistics
max_distance_value = distance_image_aoi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi_rectangle,
    scale=target_scale,
    maxPixels=1e13
).get('distance_m')
# Convert to kilometers for easier interpretation
max_distance_value_km = ee.Number(max_distance_value).divide(1000)
print(f"Maximum distance value in the image: {max_distance_value_km.getInfo()} km")

In [ ]:
# --- 6. Final Plot (Distance within AOI) ---
print(f"\n--- Plotting Final Distance Map ({aoi_bbox_radius_km*2}km AOI Rectangle) ---")
map_final = geemap.Map()
map_final.centerObject(aoi_rectangle, 6) # Center map on the AOI

# Define visualization parameters for distance
dist_vis_params = {
    'min': 0,
    'max': max_search_distance_meters, # Visualize up to the max search distance
    'palette': ['0000FF', '00FFFF', 'FFFF00', 'FF0000'] # Blue (near) to Red (far)
}

# Add the AOI bounding box outline for reference
map_final.addLayer(
    aoi_rectangle,
    {'color': 'yellow', 'fillColor': '00000000'}, # Make fill transparent
    f'{aoi_bbox_radius_km*2}km AOI Bounding Box',
    True # Keep visible
)

# --- Reproject for Visualization ---
# Reproject the distance image to WGS84 (EPSG:4326) ONLY for map display [1, 2]
# This does not change the underlying 'distance_image_aoi' variable used for calculation.
# distance_image_viz = distance_image_aoi.reproject(
#   crs='EPSG:3857', # Switch to Web Mercator for visualization
#   scale=target_scale # Keep original scale for reprojection basis
# )
# print("Reprojecting distance map to WGS84 for visualization purposes...")

# Add the REPROJECTED final distance map
map_final.addLayer(
    # distance_image_viz, # Use the WGS84 reprojected version for display
    distance_image_aoi,
    dist_vis_params,
    f'Distance (m) - {aoi_bbox_radius_km*2}km AOI (WGS84 Viz)' # Updated layer name
)

# Add a color bar legend
map_final.add_colorbar(dist_vis_params, label="Distance to Nearest Point (m)")

display(map_final)

print("\n--- Workflow Complete ---")
print("The final map shows the distance (in meters) to the nearest soil sample point,")
print(f"calculated *only* within the {aoi_bbox_radius_km*2}km x {aoi_bbox_radius_km*2}km AOI rectangle centered on Brazil.")
print("Note: The map layer is displayed reprojected to WGS84 for compatibility.")

In [ ]:
# --- Define Export Parameters ---
export_params = {
    'image': distance_image_viz,
    'description': 'ASR distance_image_export_to_drive', # A unique name for the task
    'folder': 'Earth Engine Exports',  # Optional: The folder in your Google Drive
    'fileNamePrefix': 'distance_image_aoi', # Optional: Prefix for the filename
    'region': aoi_rectangle,          # The geometry defining the export area
    # 'scale': 30,                     # Resolution in meters. Adjust as needed!
    'scale': distance_image_aoi.projection().nominalScale(), # Optional: Use the image's native scale
                                     # Use image.projection().nominalScale()
                                     # if you want the native resolution.
    'crs': 'EPSG:4326',              # Optional: Coordinate Reference System (e.g., WGS84)
    'maxPixels': 1e10                # Optional: Increase if needed for large exports
    # 'fileFormat': 'GeoTIFF'        # Optional: Default is GeoTIFF, others like 'TFRecord' available
}

# --- Start the Export Task ---
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

# --- Monitor the Task ---
print(f"Export task started. Task ID: {task.id}")
print("You can monitor the task status in the Google Earth Engine Code Editor 'Tasks' tab,")
print("or check task.status() periodically.")

# Example of checking status (optional):
# import time
# while task.active():
#   print('Polling for task (id: {}).'.format(task.id))
#   time.sleep(30)
# print('Task status:', task.status())




In [ ]:
task.status()